In [ ]:
import os  # 운영 체제와 상호작용하기 위한 모듈
import shutil  # 파일 및 디렉토리 작업을 위한 모듈
import random  # 무작위 작업을 위한 모듈
import scipy  # 과학 및 기술 계산을 위한 모듈
import numpy as np  # 수치 연산을 위한 Python 라이브러리
import matplotlib.pyplot as plt  # 데이터 시각화를 위한 라이브러리
import tensorflow as tf  # 딥러닝을 위한 TensorFlow 라이브러리
from tensorflow import keras  # TensorFlow의 고수준 API
from keras import layers  # Keras의 레이어 클래스
from keras.preprocessing.image import ImageDataGenerator  # 이미지 데이터 전처리 및 증강

def generators(train_dir, val_dir, size=64, image_size=224):
    """학습 및 검증 데이터를 생성하는 함수"""
    train_datagen = ImageDataGenerator(rescale=1/255,  # 이미지의 픽셀 값을 0-1 범위로 정규화
                                       rotation_range=180,  # 회전 범위 설정
                                       width_shift_range=0.2,  # 가로 방향 이동 범위
                                       height_shift_range=0.2,  # 세로 방향 이동 범위
                                       shear_range=0.2,  # 시어 변환 범위
                                       zoom_range=0.2,  # 줌 인/줌 아웃 범위
                                       horizontal_flip=True,  # 가로 방향 뒤집기 활성화
                                       vertical_flip=True,  # 세로 방향 뒤집기 활성화
                                       fill_mode='nearest')  # 빈 공간 채우기 모드 설정

    val_datagen = ImageDataGenerator(rescale=1/255)  # 검증 데이터의 픽셀 값을 0-1 범위로 정규화

    # 학습 데이터 생성기
    train_generator = train_datagen.flow_from_directory(train_dir,
                                                        target_size=(image_size, image_size),  # 입력 이미지 크기 설정
                                                        batch_size=size,  # 배치 크기 설정
                                                        class_mode='categorical',  # 다중 클래스 분류 설정
                                                        shuffle=True)  # 데이터 섞기 활성화

    # 검증 데이터 생성기
    val_generator = val_datagen.flow_from_directory(val_dir,
                                                    target_size=(image_size, image_size),  # 입력 이미지 크기 설정
                                                    batch_size=size,  # 배치 크기 설정
                                                    class_mode='categorical')  # 다중 클래스 분류 설정

    return train_generator, val_generator  # 학습 및 검증 데이터 생성기 반환

# 데이터 디렉토리 설정 (경로는 your_path로 대체)
train_dir = 'your_path'
val_dir = 'your_path'

# 데이터 생성기 호출
train_generator, val_generator = generators(train_dir, val_dir)

def create_patches(images, patch_size):
    """이미지를 패치로 분할하는 함수"""
    patches = tf.image.extract_patches(
        images=images,
        sizes=[1, patch_size, patch_size, 1],  # 패치 크기 설정
        strides=[1, patch_size, patch_size, 1],  # 패치 이동 거리 설정
        rates=[1, 1, 1, 1],  # 패치 추출 속도 설정
        padding='VALID')  # 패딩 설정
    patches = tf.reshape(patches, [tf.shape(images)[0], -1, patch_size * patch_size * 3])  # 패치 재구성
    return patches

class patchEncoder(layers.Layer):
    """패치 인코더 레이어 클래스"""
    def __init__(self, num_patches, projection_dim):
        super(patchEncoder, self).__init__()
        self.num_patches = num_patches  # 패치 수
        self.projection = layers.Dense(units=projection_dim)  # 프로젝션 레이어
        self.position_embedding = layers.Embedding(input_dim=num_patches, output_dim=projection_dim)  # 위치 임베딩

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)  # 패치 위치 계산
        positions = tf.expand_dims(positions, axis=0)  # 차원 확장
        encoded = self.projection(patch) + self.position_embedding(positions)  # 프로젝션 및 위치 임베딩 합산
        return encoded

    def get_config(self):
        config = super(patchEncoder, self).get_config()
        config.update({'num_patches': self.num_patches, 'projection_dim': self.projection.units})  # 설정 업데이트
        return config

class MultiHeadAttention(layers.Layer):
    """멀티 헤드 어텐션 레이어 클래스"""
    def __init__(self, num_heads, key_dim):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads  # 헤드 수
        self.key_dim = key_dim  # 키 차원
        self.projection_dim = key_dim // num_heads  # 프로젝션 차원

        self.query_dense = layers.Dense(key_dim)  # 쿼리 밀집 레이어
        self.key_dense = layers.Dense(key_dim)  # 키 밀집 레이어
        self.value_dense = layers.Dense(key_dim)  # 값 밀집 레이어
        self.combine_heads = layers.Dense(key_dim)  # 헤드 결합 레이어

    def split_heads(self, inputs, batch_size):
        """입력을 여러 헤드로 분할하는 함수"""
        inputs = tf.reshape(inputs, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])  # (batch_size, num_heads, seq_length, projection_dim) 반환

    def call(self, query, key, value):
        batch_size = tf.shape(query)[0]

        query = self.query_dense(query)  # 쿼리 프로젝션
        key = self.key_dense(key)  # 키 프로젝션
        value = self.value_dense(value)  # 값 프로젝션

        query = self.split_heads(query, batch_size)  # 쿼리 헤드 분할
        key = self.split_heads(key, batch_size)  # 키 헤드 분할
        value = self.split_heads(value, batch_size)  # 값 헤드 분할

        score = tf.matmul(query, key, transpose_b=True)  # 점수 계산
        score = score / tf.math.sqrt(tf.cast(self.projection_dim, tf.float32))  # 스케일링
        weights = tf.nn.softmax(score, axis=-1)  # 소프트맥스 계산
        attention = tf.matmul(weights, value)  # 어텐션 값 계산

        attention = tf.transpose(attention, perm=[0, 2, 1, 3])  # 어텐션 결과 변환
        concat_attention = tf.reshape(attention, (batch_size, -1, self.key_dim))  # 헤드 결합

        outputs = self.combine_heads(concat_attention)  # 최종 결과
        return outputs

def visionTransformer(input_shape,
                      patch_size,
                      num_patches,
                      projection_dim,
                      num_heads,
                      num_transformer_layers,
                      mlp_head_units,
                      dropout_rate=.1,
                      num_classes=10):
    """비전 트랜스포머 모델 정의 함수"""
    inputs = layers.Input(shape=input_shape)
    patches = create_patches(inputs, patch_size)
    encoded_patches = patchEncoder(num_patches, projection_dim)(patches)

    for _ in range(num_transformer_layers):
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)  # 레이어 정규화
        attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim)(x1, x1, x1)  # 어텐션 레이어
        x2 = layers.Add()([attention_output, encoded_patches])  # 스킵 연결
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)  # 레이어 정규화
        x3 = layers.Dense(units=projection_dim, activation=tf.nn.gelu)(x3)  # MLP
        x3 = layers.Dropout(dropout_rate)(x3)  # 드롭아웃
        x3 = layers.Dense(units=projection_dim, activation=tf.nn.gelu)(x3)  # MLP
        x3 = layers.Dropout(dropout_rate)(x3)  # 드롭아웃
        encoded_patches = layers.Add()([x3, x2])  # 스킵 연결

    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)  # 최종 레이어 정규화
    representation = layers.GlobalAveragePooling1D()(representation)  # 전역 평균 풀링

    features = layers.Dense(units=mlp_head_units[0], activation=tf.nn.gelu)(representation)  # MLP 헤드
    for units in mlp_head_units[1:]:
        features = layers.Dense(units=units, activation=tf.nn.gelu)(features)
    logits = layers.Dense(100)(features)  # ImageNet 100 클래스 예측

    model = tf.keras.Model(inputs=inputs, outputs=logits)  # 모델 생성
    return model

# 하이퍼파라미터 설정
input_shape = (224, 224, 3)
patch_size = 16
num_patches = (input_shape[0] // patch_size) ** 2
projection_dim = 64
num_heads = 4
num_transformer_layers = 8
mlp_head_units = [2048, 1024]

# 모델 생성
vit = visionTransformer(input_shape=input_shape,
                        patch_size=patch_size,
                        num_patches=num_patches,
                        projection_dim=projection_dim,
                        num_heads=num_heads,
                        num_transformer_layers=num_transformer_layers,
                        mlp_head_units=mlp_head_units,
                        dropout_rate=.3,
                        num_classes=100)

# 모델 컴파일
vit.compile(optimizer=keras.optimizers.Adam(learning_rate=3e-3, decay=3e-1),
            loss=keras.losses.CategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])

# 체크포인트 및 조기 종료 설정 (경로는 your_path로 대체)
model_path = 'your_path'
check_point = keras.callbacks.ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 모델 학습
history = vit.fit(train_generator, batch_size=512, epochs=10000, validation_data=val_generator, callbacks=[check_point, early_stopping])

# 학습 결과 시각화
plt.plot(history.history['loss'])
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_loss'])
plt.plot(history.history['val_accuracy'])
plt.title('Vision Transformer ImageNet 100')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'accuracy', 'val_loss', 'val_accuracy'], loc='upper right')
plt.show()

# 최고 검증 정확도 및 손실 출력
print('Best Val Acc:', max(history.history['val_accuracy']))
print('Best Val Loss:', min(history.history['val_loss']))